# 한국어 위키백과 파일 사용

In [1]:
import os
import re
from tqdm.auto import tqdm
from tokenizers import BertWordPieceTokenizer
import logging

datasetsFilePath = "datasets/"

In [2]:
def text_preprocess_kor_for_wordpiece(text : str) :
    text = str(text)

    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'\S*@\S*\s?', '', text)
    text = re.sub(r'<[^>]*>', '', text)
    
    target = r"[^가-힣0-9a-zA-Z.,!?'\" ]"
    text = re.sub(target, repl=" ", string=text)

    text = re.sub(r'([.,!?"\'])(\1{1,})', r'\1', text)

    text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ])\1+', r'\1', text)

    text = re.sub(r'\d+', '', text)

    text = re.sub(r"\s+", repl=" ", string=text)
    
    return text.strip()

In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
input_exracted_dir = f'{datasetsFilePath}extracted_wiki_text_gensim'
output_preprocessed_dir = f'{datasetsFilePath}preprocess_wiki_text'

if not os.path.exists(output_preprocessed_dir):
    os.makedirs(output_preprocessed_dir)

logging.info(f"'{input_exracted_dir}' 디렉토리에서 텍스트 전처리 시작")

total_processed_files = 0
total_processed_lines = 0

for root, _, files in os.walk(input_exracted_dir):
    for filename in files:
        if filename.endswith('.txt'):
            input_file_path = os.path.join(root,filename)
            output_file_path = os.path.join(output_preprocessed_dir,filename)

            logging.info(f"'{input_file_path}' 파일 전처리 중")
            processed_lines_in_file = 0
            with open(input_file_path, 'r', encoding='utf-8') as infile, \
                 open(output_file_path, 'w', encoding='utf-8') as outfile:
                progress_bar = tqdm(infile)
                for line in progress_bar:
                    preprocessed_line = text_preprocess_kor_for_wordpiece(line.strip())
                    if preprocessed_line:
                        outfile.write(preprocessed_line+'\n')
                        processed_lines_in_file += 1
            total_processed_files += 1
            total_processed_lines += processed_lines_in_file
            logging.info(f"'{input_file_path}' 전처리 완료. {processed_lines_in_file}줄 저장됨.")
logging.info(f"\n--- 텍스트 전처리 완료 ---")
logging.info(f"총 {total_processed_files}개 파일에서 {total_processed_lines}줄이 전처리되어 '{output_preprocessed_dir}'에 저장됨.")

2025-07-04 22:52:18,779 : INFO : 'datasets/extracted_wiki_text_gensim' 디렉토리에서 텍스트 전처리 시작
2025-07-04 22:52:18,781 : INFO : 'datasets/extracted_wiki_text_gensim\TS_액티벤처.txt' 파일 전처리 중


0it [00:00, ?it/s]

2025-07-04 22:52:18,910 : INFO : 'datasets/extracted_wiki_text_gensim\TS_액티벤처.txt' 전처리 완료. 9881줄 저장됨.
2025-07-04 22:52:18,910 : INFO : 'datasets/extracted_wiki_text_gensim\TS_엘지유플러스.txt' 파일 전처리 중


0it [00:00, ?it/s]

2025-07-04 22:52:20,001 : INFO : 'datasets/extracted_wiki_text_gensim\TS_엘지유플러스.txt' 전처리 완료. 134189줄 저장됨.
2025-07-04 22:52:20,001 : INFO : 'datasets/extracted_wiki_text_gensim\TS_하나카드.txt' 파일 전처리 중


0it [00:00, ?it/s]

2025-07-04 22:52:21,759 : INFO : 'datasets/extracted_wiki_text_gensim\TS_하나카드.txt' 전처리 완료. 216609줄 저장됨.
2025-07-04 22:52:21,759 : INFO : 'datasets/extracted_wiki_text_gensim\TS_한국어말뭉치데이터_구어체.txt' 파일 전처리 중


0it [00:00, ?it/s]

2025-07-04 23:00:14,733 : INFO : 'datasets/extracted_wiki_text_gensim\TS_한국어말뭉치데이터_구어체.txt' 전처리 완료. 28730474줄 저장됨.
2025-07-04 23:00:14,733 : INFO : 
--- 텍스트 전처리 완료 ---
2025-07-04 23:00:14,733 : INFO : 총 4개 파일에서 29091153줄이 전처리되어 'datasets/preprocess_wiki_text'에 저장됨.


In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

preprocessed_text_dir = f'{datasetsFilePath}preprocess_wiki_text'
output_tokenizer_dir = f'{datasetsFilePath}custom_tokenizer_output' 

if not os.path.exists(output_tokenizer_dir):
    os.makedirs(output_tokenizer_dir)

files = [os.path.join(preprocessed_text_dir, f) for f in os.listdir(preprocessed_text_dir) if f.endswith('.txt')]

if not files:
    logging.error(f"'{preprocessed_text_dir}' 디렉토리에 전처리된 텍스트 파일이 없습니다. 1단계 전처리 단계를 먼저 완료하세요.")
else:
    logging.info(f"총 {len(files)}개의 전처리된 파일을 사용하여 WordPiece 어휘집 학습 시작...")

    tokenizer = BertWordPieceTokenizer(
        clean_text=True,
        handle_chinese_chars=False,
        strip_accents=False,
        lowercase=False
    )

    tokenizer.train(
        files,
        vocab_size=32000,
        min_frequency=5,
        show_progress=True,
        special_tokens=[
            "[PAD]",
            "[UNK]",
            "[CLS]", 
            "[SEP]",
            "[MASK]"
        ]
    )

    tokenizer.save_model(output_tokenizer_dir)

    logging.info(f"WordPiece 어휘집 학습 완료. '{output_tokenizer_dir}/vocab.txt'에 저장되었습니다.")
    logging.info(f"생성된 어휘집 크기: {tokenizer.get_vocab_size()}")

2025-07-04 23:01:26,188 : INFO : 총 4개의 전처리된 파일을 사용하여 WordPiece 어휘집 학습 시작...
2025-07-04 23:04:07,029 : INFO : WordPiece 어휘집 학습 완료. 'datasets/custom_tokenizer_output/vocab.txt'에 저장되었습니다.
2025-07-04 23:04:07,035 : INFO : 생성된 어휘집 크기: 32000
